In [ ]:
import dashscope
import os
import json
import requests
import jsonlines
from tqdm import tqdm
import time
from multiprocessing import Pool
from functools import partial

from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    RetryError
)

In [ ]:
data = list(jsonlines.open("../data/wiki_roleplay_multilingual_test_input_w_evidence.jsonl"))

In [ ]:
def call_qwen(messages):
    try:
        response = dashscope.Generation.call(
            model='qwen-plus', # or qwen-max
            presense_penalty=0,
            messages=messages,
            result_format='message',
            headers={"X-DashScope-DataInspection": "disable"}
        )
        assert response is not None and response.output is not None
        return response
    except Exception as e:
        pass

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(5))
def process(item):
    dashscope.api_key = "your api key"
    messages = [{'role': 'system', 'content': item['system']}]
    for prompt in item['prompts']:
        messages.append({"role": "user", "content": prompt})
        response = call_qwen(messages)
        messages.append(dict(response.output.choices[0].message))
        time.sleep(1/60)
    item['messages'] = messages
    return item

In [ ]:
results = []
pbar = tqdm(total=len(data))
for item in data:
    results.append(process(item))
    pbar.update(1)

In [ ]:
with jsonlines.open(f"../data/results/wiki_roleplay_multilingual_test_input_w_evidence_qwen-plus_output.jsonl", "w") as f:
    for each in results:
        f.write(each)